In [ ]:
import os
import pandas as pd
'''Here the library panda is recalled pd for semplicity'''
from tqdm.notebook import tqdm
''' We use only the notebook part from tqdm library'''
tqdm().pandas()

import json
import numpy as np
import shutil

import spacy
nlp = spacy.load('en_core_web_sm')
'''we define a nlp object loading a small english dictionary '''

TAG_MAP = ['SYM',
 'PROPN',
 'DET',
 'VERB',
 'CCONJ',
 'INTJ',
 'ADV',
 'ADJ',
 'PUNCT',
 'NOUN',
 'PRON',
 'PART',
 'X',
 'NUM',
 'SPACE',
 'ADP']
'''every element in our dictionary is tag with these labels shown (SYM, PROPN, DET...ADP)'''

'''patternize is a function that transform 'x' in pattern. 'x' is the clue reported in the second column of .csv file'''
def patternize(x, label):
        
    output = {}
    
    output['label'] = label
    
    pattern = []
    for t in x.split(' '):
        
        token = {}
        token['LEMMA'] = t.lower()
        pos = {}
        pos['NOT_IN'] = ['VERB']
        token['POS'] = pos
        
        pattern.append(token)
        
    output['pattern'] = pattern
    
    return output

'''The function defined below (excelize) is not used
def excelize(pattern_list):
    
    patterns_df = []
    
    for pattern in pattern_list:
        
        label = pattern['label']
        
        chunks = []
        for chunk in pattern['pattern']:
            for item in chunk.items():
                if item[0] == 'LEMMA':
                    chunks.append(item[1])
                else:
                    chunks.append(item[0].upper() + '_' + item[1])
                
        patterns_df.append([label, ' '.join(chunks)])
        
    return pd.DataFrame(patterns_df,columns = ['label','name'])'''


from spacy.tokens import Span
from spacy.language import Language

stops = ['the','many','some','this','a','an','various','different','same', 'any', 'that','its', 'common', 'useful', 'another', 'such a', 'such', 'these', 'those', 'their']

@Language.component("expand_entities")
def expand_entities(doc):
    
    new_ents = []
    
    for ent in doc.ents:
        span = [ent.start, ent.end]
    
        for t in ent.lefts:    
            if t.pos_ != 'VERB' and t.lower_ not in stops:
                span.append(t.i)
                
        new_ent = Span(doc, min(span), max(span), label = ent.label)
        new_ents.append(new_ent)
    
    
    doc.ents = spacy.util.filter_spans(new_ents)
        
    return doc

@Language.component("extract_from_clues")
def extract_from_clues(doc):
    
    new_ents = []
    
    for ent in doc.ents:

        if 'CLUE' in ent.label_:
            
            span = []
            rights = list(ent.rights)

            span = []
            while rights != []:
                for right in rights:
                    #if (right.dep_ != 'prep') & (right.dep_ != 'punct') & (right.dep_ != 'advcl') & (right.dep_ != 'ccomp'):

                    if (right.dep_ != 'punct') & (right.dep_ != 'advcl') & (right.dep_ != 'ccomp'):

                    #while True:
                        rights.remove(right)
                        rights.extend(list(right.rights))
                        span.append(right.i)
                    else:
                        rights.remove(right)

            try:             
              new_ent = Span(doc, min(span), max(span)+1, label = ent.label_.split('|')[1])
              new_ents.append(new_ent)
            except Exception:
              continue
             
        
        else:
            new_ents.append(ent)
    
    
    doc.ents = spacy.util.filter_spans(new_ents)
    
    return doc

from spacy.language import Language

patterns = pd.read_csv('./clues.csv', sep = ';')

with open('./patterns.jsonl', 'w') as outfile:
    for entry in patterns.progress_apply(lambda x: patternize(x['name'],x['label']), axis = 1):
        json.dump(entry, outfile)
        outfile.write('\n')
        
from spacy.pipeline import EntityRuler

nlp = spacy.load("en_core_web_sm")
nlp.remove_pipe('ner')
nlp.add_pipe("entity_ruler").from_disk('./patterns.jsonl')


@Language.component("set_custom_boundaries")
def set_custom_boundaries(doc):
    for token in doc[:-1]:
        if token.text == ";":
            doc[token.i+1].is_sent_start = True
    return doc

nlp.add_pipe('set_custom_boundaries', before="parser")

#nlp.add_pipe('expand_entities')
nlp.add_pipe('extract_from_clues')


def skill_clue_extract(text, clues_path = './clues.csv'):
  '''This function takes as input a string and returns the list of skills that follow certain clues that are defined in the csv file called `clues.csv`'''
  doc = nlp(text) 
  
  return [(e.text) for e in doc.ents]

0it [00:00, ?it/s]

  0%|          | 0/22 [00:00<?, ?it/s]

# Steps
## Input
- File csv con due colonne:
  - Colonna "label": "CLUE|SKILL"
  - Colonna "name": _clue (e.g. ability to, capability to, etc..._)

In [ ]:
text = 'Organizations strive to develop a variety of ability to improve and measure business processes.'
#text = 'talking is a great ability'
#patternize("ability of", "CLUE|SKILL")
skill_clue_extract(text)


['improve and measure business processes']